In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import os
import json

In [11]:
# Dataset path
DATASET_DIR = "dataset"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5  # You can increase later if needed

In [12]:
# Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [13]:
from PIL import Image
import os

def remove_broken_images(folder_path):
    removed = 0
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                img = Image.open(file_path)
                img.verify()  # This will raise an exception for corrupt images
            except Exception as e:
                if os.path.exists(file_path):  # Check if the file exists
                    print(f"Removing broken image: {file_path}")
                    os.remove(file_path)
                else:
                    print(f"File not found, skipping: {file_path}")
                removed += 1
    print(f"✅ Removed {removed} broken images.")

remove_broken_images("dataset")


✅ Removed 0 broken images.


In [14]:
# ── Create the training generator ────────────────────────────────────────────
train_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    subset='training'
)

Found 557 images belonging to 9 classes.


In [15]:
# ── SAVE the mapping from class name → index ─────────────────────────────────
os.makedirs("models", exist_ok=True)               # ensure folder exists
with open("models/skin_class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f)
print("Saved class_indices mapping:", train_generator.class_indices)

Saved class_indices mapping: {'Actinic keratosis': 0, 'Atopic Dermatitis': 1, 'Benign keratosis': 2, 'Dermatofibroma': 3, 'Melanocytic nevus': 4, 'Melanoma': 5, 'Squamous cell carcinoma': 6, 'Tinea Ringworm Candidiasis': 7, 'Vascular lesion': 8}


In [16]:
# Validation generator
val_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 139 images belonging to 9 classes.


In [18]:
# Base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train!
model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

# Save model
model.save("models/skin_model.keras")
print("✅ Model saved at models/skin_model.h5")


Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 791ms/step - accuracy: 0.2309 - loss: 2.1750 - val_accuracy: 0.4964 - val_loss: 1.4989
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 672ms/step - accuracy: 0.5756 - loss: 1.2587 - val_accuracy: 0.6259 - val_loss: 1.1496
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 653ms/step - accuracy: 0.6929 - loss: 0.9848 - val_accuracy: 0.6547 - val_loss: 0.9754
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 621ms/step - accuracy: 0.7501 - loss: 0.7803 - val_accuracy: 0.6763 - val_loss: 0.9172
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 619ms/step - accuracy: 0.7630 - loss: 0.7322 - val_accuracy: 0.6691 - val_loss: 0.8837
✅ Model saved at models/skin_model.h5
